In [1]:
import sys
from pathlib import Path

main_path = Path('..').resolve()
sys.path.append(str(main_path))

from src.dataset import MetaStockDataset
from src.utils import ARGProcessor
import numpy as np

In [44]:
setting_file = Path('.') / 'kdd.yml'

meta_args = ARGProcessor(setting_file=setting_file)
data_kwargs = meta_args.get_args(cls=MetaStockDataset)

In [45]:
data_dir = data_kwargs['data_dir']
dtype = data_kwargs['dtype']
n_train_stock = data_kwargs['n_train_stock']

In [46]:
data_dir = Path(data_dir).resolve()
ds_info = {
    # train: (Jan-01-2007 to Jan-01-2015)
    # val: (Jan-01-2015 to Jan-01-2016)
    # test: (Jan-01-2016 to Jan-01-2017)
    'kdd17': {
        'path': data_dir / 'kdd17/price_long_50',
        'date': data_dir / 'kdd17/trading_dates.csv',
        'train_date': '2015-01-01', 
        'val_date': '2016-01-01', 
        'test_date': '2017-01-01',
    },
    # train: (Jan-01-2014 to Aug-01-2015)
    # vali: (Aug-01-2015 to Oct-01-2015)
    # test: (Oct-01-2015 to Jan-01-2016)
    'acl18': {
        'path': data_dir / 'stocknet-dataset/price/raw',
        'date': data_dir / 'stocknet-dataset/price/trading_dates.csv',
        'train_date': '2015-08-01', 
        'val_date': '2015-10-01', 
        'test_date': '2016-01-01',
    }
}
ds_config = ds_info[dtype]

meta_type = 'train'
window_sizes = [5] # [5, 10, 15, 20]

# get data
data = {}
candidates = {}
ps = list((data_dir / ds_config['path']).glob('*.csv'))
# iterator = ps[:n_train_stock] if (meta_type == 'train') or (meta_type == 'test1') else ps[n_train_stock:]


In [47]:
n_stocks = len(ps)
print(n_stocks)

50


In [48]:
seeds = [3, 7, 11, 69, 81]
all_selected = []
all_not_selected = []
for s in seeds:
    np.random.seed(s)
    selected = np.random.choice(ps, size=int(n_stocks*0.8), replace=False)
    not_selected = np.array(ps)[~np.isin(ps, selected)]
    all_selected.append([s.name.strip('.csv') for s in selected])
    all_not_selected.append([s.name.strip('.csv') for s in not_selected])

In [49]:
import json

d = {}

for i, s in enumerate(seeds):
    d[i] = {
        'seed': s, 'known': all_selected[i], 'unknown': all_not_selected[i]
    }
with Path('stock_universe.json').open('w', encoding='utf-8') as file:
    json.dump(d, file)